# Power scraper
The power scraper's goal is to scrape hourly power production averages from [ENTSO-E](https://en.wikipedia.org/wiki/European_Network_of_Transmission_System_Operators_for_Electricity)'s [API](https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html), and put it into a pandas dataframe for smoother analysis going onwards.

From [this PDF (page 13/16)](https://transparency.entsoe.eu/content/static_content/download?path=/Static%20content/web%20api/RestfulAPI_IG.pdf) I've learned that we are looking to scrape a _Document Type_ named _A73_ and described as _Actual generation output per generation unit_. This document type is also coupled with a _Process Type_ named _A16_ and described as _Realised_ that I believe reference that it relates to actual generation as compared to for example forcasted generation.

### References
- [ENTSO-E API implementation guide](https://transparency.entsoe.eu/content/static_content/download?path=/Static%20content/web%20api/RestfulAPI_IG.pdf)
- [ENTSO-E API web guide](https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html)
- [EnergieID/entsoe-py](https://github.com/EnergieID/entsoe-py): A python client for working against the ENTSO-E API
- [ElectricityMap's use of ENTSO-E's API](https://github.com/tmrowco/electricitymap-contrib/blob/master/parsers/ENTSOE.py)

### Setup to run code
In order for you to be able to use this code, you need an API Token for accessing the ENTSO-E API. For instructions on how to get one, see [their documentation about it](https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_authentication_and_authorisation).

This is what you need to do summarized:

1. Create an account on the [Transparency Platform (TP)](https://transparency.entsoe.eu/).
2. Send an email to transparency@entsoe.eu with `Restful API access` in the subject line. Indicate the email address you entered during registration in the email body.
3. Await a response and inspect [your account settings](https://transparency.entsoe.eu/usrm/user/myAccountSettings) where a _Web Api Security Token_ should now be available.
4. Write your API_TOKEN to the `.env` file or set manually update your environment variable `API_TOKEN` for use by the scripts below. The file should contain one line of text formatted like this:

   ```
   API_TOKEN=your-api-token-here
   ```

In [ ]:
# Load's API_TOKEN from the .env file which is .gitignore'd
from dotenv import load_dotenv
load_dotenv()

## Code strategy
We rely heavily on the `EntsoePandasClient` of the `entsoe` Python library, and its [query_generation_per_plant function](https://github.com/EnergieID/entsoe-py/blob/e620357ad6ea0ddd217d4cff61eed18e8461f584/entsoe/entsoe.py#L905). Iterating over it for each country, and using a time interval from the beginning of the API's creation, to now.

Caching web requests, saving completed country downloads, and progressbars is the glue that takes us all the way.

### About web request caching
We should avoid making web request to the API in order to avoid paing a cost of time and potentially getting rate limited or banned. To do this we can try to cache as much requests we make as possible, as the data isn't supposed to change anyhow.

Consider us using the `query_generation_per_plant` function and passing a timespan of a week. I've learned it will make seven requests to the ENTSO-E API, one per day, because it is simply a restriction on their API. If we now specify overlapping days in a later request, these days will be cached and we will only probe the actual API with those that had not already been requested. All of this is done by us importing [requests-cache](https://github.com/reclosedev/requests-cache) and saying we want to use it.

In [2]:
# use a cache so we don't ask the same thing twice
# this is what is stored in the entso-e.sqlite database file
import requests_cache

### Progress bars for sanity
When working with long running jobs, having a progress bar is very useful. The tqdm library provides such functionality.

In [3]:
# progress bars beauty
# https://github.com/tqdm/tqdm#ipythonjupyter-integration
from tqdm.auto import tqdm

### Modifications to entsoe library
I made some modification to not abort scraping if a exception about no available data was thrown for an individual day. This changes are part of a fork on https://github.com/consideRatio/entsoe-py.

### Code improvements
...

## Scraping time estimates
It took 130 seconds to get data about Sweden for a month, or about five seconds per day. The dates to investigate range from 2015-01-05 onwards, which is about five years, which is about 60 months. 60 months * 130 seconds / month = 130 minutes for one country. With 42 different country entries we can choose from in the `entsoe.mappings.DOMAIN_MAPPINGS`, we have something that needs to run for about 130 minutes * 42 ~= 3.8 days.

# Scraper

In [4]:
from entsoe import EntsoePandasClient
from entsoe.mappings import DOMAIN_MAPPINGS
import entsoe.misc

import os
import pandas as pd

In [5]:
def get_country_data(country_code, start, end):
    """
    Returns a pandas dataframe containing the specified country's
    production units and how much energy they produced that hour
    with a hourly resolution.
    """
    # To help us not clutter the output of the cell
    from IPython.utils import io
    
    # Return previously downloaded dataframes
    dataframe_file = f"dataframes/{country_code.lower()}.pickle"
    if os.path.isfile(dataframe_file):
        print(f"Skipping download, a dataframe was already found for {country_code}.")
        return pd.read_pickle(dataframe_file)
    
    # Use a cache for this country
    cache_file = f"caches/{country_code.lower()}"
    requests_cache.install_cache(cache_file)
    
    # This will do the heavy lifting
    client = EntsoePandasClient(
        api_key=os.environ["API_TOKEN"],
        session=None,
        retry_count=3,
        retry_delay=0,
        proxies=None,
    )
    
    # Create monthly blocks so we can have a nice progressbar the
    # increments with each month. The requests will still be made
    # on a daily basis by the entsoe library, as required by the
    # ENTSO-E API.
    month_intervals = list(entsoe.misc.month_blocks(start, end))
    
    # Start scraping using the entsoe library acting as a helper
    # python API to communicate with the ENTSO-E API.
    dfs = []
    for month_start_datetime, month_end_datetime in tqdm(month_intervals, desc=country_code):
        with io.capture_output() as captured_output:
            print(f"Scraping month: {month_start_datetime.year}-{month_start_datetime.month:02d}")
            dfs.append(
                client.query_generation_per_plant(
                    country_code,
                    start=month_start_datetime,
                    end=month_end_datetime,
                )
            )
    
    # Combine the scraped monthly dataframes into one large
    df = pd.concat(dfs)
    
    # Save the combined dataframe to disk as a pickle file
    # NOTE: the pickle file for Sweden was 19 MB, while the
    # request cache was 335 MB.
    df.to_pickle(dataframe_file)
    
    # return the dataframe
    return df

In [ ]:
# NOTE: the ENTSO-E API doesn't have data before 2015
# NOTE: I think the API is pretty much hardcoded for use with the
# Europe/Brussels timezone. There could be daylight savings time
# matters that can mess with an hour of data or two during the year.
api_data_start = pd.Timestamp(year=2015, month=1,  day=1,  tz='Europe/Brussels')
api_data_end   = pd.Timestamp(year=2019, month=11, day=15, tz='Europe/Brussels')

# Iterate over the 42 domains (countries mostly), and get their data
for country_code in tqdm(DOMAIN_MAPPINGS, desc="Countries"):
    get_country_data(country_code, api_data_start, api_data_end)